# Introduction to Machine Learning

D4G workshop

Potsdam, 13.06.22

Author: Caroline Arnold, DKRZ / Helmholtz AI

## Introduction

Welcome to Introduction to Machine Learning! Today we are going to work through the typical lifecycle of a machine learning project. We will be working with data from the CyGNSS satellite mission to predict global ocean wind speed.


### Setup

This tutorial could be done on a laptop, but for convenience we will use Google Colab.

1. Go to https://colab.research.google.com
2. Sign in with a google account
3. Open a new notebook (File > New notebook)
3. Clone the tutorial git repository by executing the following command (the "!" is important here!)

```bash
! git clone https://github.com/crlna16/d4g-tutorial
```

The data is stored separately in the subfolder `./data`

### Today's Goals

1. Walk through all stages of a machine learning project
1. Train a neural network 
1. Learn strategies to improve your machine learning algorithm
1. Optional: Get familiar with different neural network architectures

This notebook can serve as a reference for you to employ in your own scientific machine learning projects. We do not expect you to understand every single line of code!


<img src="./images/data-science-lifecycle8-3x.png" alt="Data Science Lifecycle" width="500" height="500" align="center">


## <font color="#0000ff" weight="bold">Scoping</font>

Before we start the machine learning project, it is important to gain an understanding of the problem at hand. 

The CyGNSS (Cyclone GNSS) is a system of microsatellites that measures GNSS reflected off the Earth's surface. We would like to predict the global ocean wind speed from CyGNSS measurements. This is a *regression problem* (the wind speed is a continuous variable). As opposed to a *classification problem* (e.g., distinguish pictures of cats and dogs).

Data is available from http://podaac.jpl.nasa.gov. We have downloaded and processed a subset of CyGNSS data for the purpose of this tutorial. This dataset contains labels from ERA5 reanalysis data, i.e., we are dealing with *supervised learning*. Other, more advanced, machine learning methods are *unsupervised learning* and *reinforcement learning*.

<img src="./images/cygnss-from-space.png" alt="CyGNSS satellites and transmitter on top of a cyclone" title="CyNGSS satellites, Image courtesy of Milad Asgarimehr">

## <font color="#0099cc">Data cleaning</font>

While it may be easy to obtain a lot of data, it is necessary to ensure the data quality, eg by checking for `None` values in the data. For the purpose of this tutorial, you can assume the data has been cleaned such that you can directly work with it.

## <font color="#33cc33">Data exploration</font>

Now it is time to take a look at the data. Check out the subdirectory `../data/`: There are three files, `train_data.nc`, `valid_data.nc`, `test_data.nc`. They can be opened with the Python library `xarray`.

In [ ]:
import xarray as xr
import numpy as np
np.random.seed(20220613)

In [ ]:
ds_train = xr.open_dataset('../data/train_data.nc')

Interactive browser for the dataset

In [ ]:
ds_train 

### Target variable

The target variable is the wind speed. To extract it:

In [ ]:
y = ds_train['windspeed'].values

Visualization is very helpful for machine learning projects, as it helps us to identify the key properties of the dataset at a glance. We plot the distribution of the wind speed:

In [ ]:
# Necessary libraries for plotting. Check out https://seaborn.pydata.org/ for reference
from matplotlib import pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
sns.set_context('notebook')

In [ ]:
sns.histplot(y)

plt.xlabel('Wind speed (m/s)')
plt.xticks([0, 2.5, 5.0, 7.5, 10, 12.5, 15, 17.5, 20])

plt.show()

### Feature variables

There are 2D and 1D variables in the dataset (use the interactive dataset browser to check that). First, we extract the 2D variables and look at some selected samples.

#### BRCS (Bistatic Radar Cross Section)

In [ ]:
brcs = ds_train['brcs'][:]

fig, ax = plt.subplots(1, 5, sharex=True, sharey=True, figsize=(20,4))

for i in range(5):
    sns.heatmap(brcs[i*100], ax=ax[i])


#### 1D variables

The dataset contains 1D variables as well. Here we can see the value ranges using histogram plots


In [ ]:
ddm_nbrcs = ds_train['ddm_nbrcs'][:]
ddm_les   = ds_train['ddm_les'][:]
sp_inc_angle = ds_train['sp_inc_angle'][:]

fig, ax = plt.subplots(1, 3, figsize=(20, 6))

sns.histplot(ddm_nbrcs, ax=ax[0])
ax[0].set_xlabel('DDM NBRCS')

sns.histplot(ddm_les, ax=ax[1])
ax[1].set_xlabel('DDM LES')

sns.histplot(sp_inc_angle, ax=ax[2])
ax[2].set_xlabel('SP INC ANGLE')

plt.show()

### Relation of feature and target variables

It is often insightful to look at density plots of feature and target variables. We do this here for the 1D variables:

In [ ]:
fig, ax = plt.subplots(1, 3, sharex=True, figsize=(20, 6))

ax[0].hexbin(y, ddm_nbrcs, mincnt=1, cmap='viridis')
ax[1].hexbin(y, ddm_les, mincnt=1, cmap='viridis')
ax[2].hexbin(y, sp_inc_angle, mincnt=1, cmap='viridis')

ax[0].set_xlabel('Wind speed (m/s)')
ax[0].set_ylabel('DDM NBRCS')

ax[1].set_xlabel('Wind speed (m/s)')
ax[1].set_ylabel('DDM LES')

ax[2].set_xlabel('Wind speed (m/s)')
ax[2].set_ylabel('SP INC ANGLE')

plt.show()

## <font color="#ffcc00">Feature engineering</font>

We can use domain knowledge to compute new features from the existing variables. This is more commonly used for *classical machine learning algorithms* such as random forests. We will skip this step here and work only with the existing features.

## <font color="#ff9900">Model development</font>

Predictive modeling includes setting up a machine learning algorithm, training it and evaluating its performance. Our algorithm of choice is a neural network. We will come to the neural network code in a minute, but first we need to prepare the data.

### Prepare the input data

The data is split into *train*, *validation*, and *test* data. All three datasets have their distinct purpose:
1. Train data is given to the machine learning algorithm to tune the parameters of the neural network
1. Validation data is used to identify when the machine learning algorithm starts to overfit to the training data (we want to avoid learning the training data by heart)
1. Test data is used to gauge the ability of an ML algorithm to generalize. This dataset was not included at all in training and validation. We set it aside for now

**Read the documentation string of the following function that we defined already for you. What does it provide?**

In [ ]:
def create_dataset(split, network='dense', normalize=True, verbose=True):
    '''
    Helper function to load the datasets that were prepared for this tutorial.
    
    Parameters:
    split       - Choice of [train, valid, test]
    network     - Choice of [dense, cnn]. Selects the appropriate 1D or 2D variables
    normalize   - Normalize features (default: True)
    verbose     - Print dataset information (default: True)
    
    Returns:
    
    (X, y) - Tuple of features and labels
    '''   
    
    if network == 'dense':
        input_keys = ['ddm_nbrcs', 'ddm_les', 'sp_inc_angle']
    elif network == 'cnn':
        input_keys = ['brcs']
    else:
        raise ValueError('Unknown option: network = ', network)
    
    ds = xr.open_dataset(f'../data/{split}_data.nc')
    
    X = []
    
    for key in input_keys:
        var = ds[key][:]
        if normalize:
            var /= np.max(var)
        X.append(var)
        
    X = np.swapaxes(np.asarray(X), 0, 1)
    
    if len(X.shape) == 4: # images to channel_last
        X = np.swapaxes(X, 1, 3)
    
    y = np.array(ds['windspeed'][:])
    y = y[:, np.newaxis]
    
    print(f'Loaded data for split {split}')
    print(f'Feature array: {X.shape}')
    print(f'Label array:   {y.shape}')
    
    return X, y

We use the function to create train and validation datasets.

In [ ]:
X_train, y_train = create_dataset('train')
X_valid, y_valid = create_dataset('valid')

### Introduction to neural networks

A single neuron takes an input $x$, applies a linear transformation $y = w \cdot x + b$, and ultimately applies a non-linear *activation function* $\sigma$, e.g., the relu function.

Therefore, a single neuron transforms the input $x$ like:

$y = \sigma( w \cdot x + b )$

The parameters $w, b$ are *learned* by exposing the neural network to training data. The dense neural network is a neural network that stacks several individual neurons together in *layers*. A forward pass through such a network can be written as 

$y = \sigma A( \sigma B (x))$

where $A, B$ are the weight matrices of the neural network.

<img src="./images/dense-neural-network.png" size="0.5">

### Define a neural network architecture

For convenience, we define a python function that can generate dense neural networks of various sizes. We use the *Keras* machine learning framework (https://keras.io/)

In [ ]:
import tensorflow.keras as keras

def create_nn(H0=16, H1=8, input_dim=(3,)):
    '''
    Create a dense neural network with two hidden layers
    
    Parameters:
    H0 - Number of neurons in 1st hidden layer
    H1 - Number of neurons in 2nd hidden layer
    input_dim - Number of input features
    '''
    
    # Create a Keras input tensor
    inputs = keras.layers.Input(shape=input_dim)
    
    # Apply the first hidden layer
    hidden_layer = keras.layers.Dense(H0, activation='relu')(inputs)
    
    # Apply the second hidden layer
    hidden_layer2 = keras.layers.Dense(H1, activation='relu')(hidden_layer)
    
    # Reduce to one final output for the regression
    outputs = keras.layers.Dense(1)(hidden_layer2) 
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    return model 

Create a model with default parameters:

In [ ]:
model = create_nn(H0=16, H1=8)

model.summary()

We also need to define an optimizer that is the strategy to reach a minimum of the neural network parameter space. In Keras, this is done by compiling the model:

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.MeanSquaredError())

### Train the neural network

In training, we show the training data to the neural network such that it can estimate the parameters. Then, the loss is calculated, here the mean squared error of true ($y$) vs predicted ($\hat y$) labels:

$\mathcal L = \frac 1 N \sum\limits_{i=1}^N (y_i - \hat y_i)^2$

Based on that, the neural network weights are adapted using backward propagation (advanced topic). We show the data to the network in *minibatches* for scalability and efficiency. We call it an *epoch of training* when we showed every training sample once to the network. Thus, we iteratively approach a set of neural network parameters that corresponds to a local minimum of the loss function.

An important question is how we should know that we should *stop* training. In theory, we could train forever and ultimately reduce the loss on the training set to 0. That would not be helpful, because the model would then not generalize well to unseen data, a phenomenon known as *overfitting*. Therefore, we monitor the loss on the validation set during training, and stop training once this loss does no longer decrease.

In [ ]:
max_epochs=200 # stop here in any case
batch_size=32  # size of the minibatch

In [ ]:
early_stopping=keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [ ]:
history = model.fit(X_train, y_train, 
                    validation_data=(X_valid, y_valid), 
                    callbacks=[early_stopping],
                    epochs=max_epochs, 
                    batch_size=batch_size)

### Analyze the training process

Plot the history of the training process. The Keras framework automatically stored the training and validation loss for each epoch.

In [ ]:
trained_epochs = len(history.history['loss'])

sns.lineplot(x=range(trained_epochs), y=history.history['loss'], label='Train loss')
sns.lineplot(x=range(trained_epochs), y=history.history['val_loss'], label='Validation loss')

plt.ylim(0, 10)

plt.xticks(range(trained_epochs))
plt.xlabel('Epoch')

plt.show()

## <font color="#ff0000">Verification</font>

Improve the neural network

### Single metric

We need to define a strategy to gauge the performance of the neural network. For that, we recommend to choose a single metric that is determined on the validation set and that you optimize step by step. In our case, this is the root mean squared error (RMSE). Calculate it below for the model we trained:

In [ ]:
from sklearn.metrics import mean_squared_error

y_pred = model.predict(X_valid)

rmse = mean_squared_error(y_valid, y_pred, squared=False)

print(f'Root mean squared error (RMSE) obtained on validation set: {rmse:.4f} m/s')

**Discuss: What would be the single metric in your machine learning project?**

## <font color="#ff3399">Workflow tuning</font>

### Next try

Change the parameters `H0, H1` of the neural network, as well as the batch size. What do you observe for the validation set results?

In [ ]:
model2 = create_nn(H0=128, H1=64)
model2.summary()

In [ ]:
model2.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.MeanSquaredError())

history = model2.fit(X_train, y_train, 
                    validation_data=(X_valid, y_valid), 
                    callbacks=[early_stopping],
                    epochs=max_epochs, 
                    batch_size=batch_size, 
                    verbose=1)

In [ ]:
y_pred = model2.predict(X_valid)

rmse = mean_squared_error(y_valid, y_pred, squared=False)

print(f'Root mean squared error (RMSE) obtained on validation set: {rmse:.4f} m/s')

Compare the RMSE to the RMSE you obtained before with the default architecture. Do you see an improvement?

**Optional: Try out more values for H0, H1, and batch_size and record your resulting RMSE**

### Advanced topic: Convolutional neural network (CNN)

Remember that the dataset contains 2D variables as well, which we did not use so far. These can be seen as images. Convolutional neural networks originated in computer vision and were originally developed for the image classification. We adapt here a convolutional neural network for regression.

<img src="./images/d4g-cnn-sketch.png">

In [ ]:
def create_cnn(n_filters=16, H0=64, H1=32):
    '''
    Create a convolutional neural network. The architecture has 2 convolutional layers, followed by two dense layers.
    
    Parameters:
    n_filters - number of filters in the convolutional layer
    H0        - number of neurons in the dense layer
    '''
    
    inputs = keras.layers.Input(shape=(11, 17, 1))
    conv_layer1 = keras.layers.Conv2D(n_filters, 3, activation="relu")(inputs)
    pooling_layer = keras.layers.MaxPool2D()(conv_layer1)
    conv_layer2 = keras.layers.Conv2D(n_filters, 3, activation="relu")(pooling_layer)
    pooling_layer2 = keras.layers.MaxPool2D()(conv_layer2)
    flatten_layer = keras.layers.Flatten()(pooling_layer2)
    dense_layer = keras.layers.Dense(H0, activation="relu")(flatten_layer)
    dense_layer2 = keras.layers.Dense(H1, activation="relu")(dense_layer)
    outputs = keras.layers.Dense(1)(dense_layer2)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    return model

We create training and validation data this time using the image data part of the provided CyGNSS dataset:

In [ ]:
X_train_cnn, _ = create_dataset('train', network='cnn')
X_valid_cnn, _ = create_dataset('valid', network='cnn')

In [ ]:
model_cnn = create_cnn(n_filters=16, H0=64, H1=32)
model_cnn.summary()

In [ ]:
model_cnn.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.MeanSquaredError())

history = model_cnn.fit(X_train_cnn, y_train, 
                    validation_data=(X_valid_cnn, y_valid), 
                    callbacks=[early_stopping],
                    epochs=max_epochs, 
                    batch_size=batch_size, 
                    verbose=1)

In [ ]:
rmse = mean_squared_error(y_valid, model_cnn.predict(X_valid_cnn), squared=False)
print(f'RMSE for the CNN: {rmse:.4f} m/s')

To summarize the results that were obtained on the *validation* set:

**Fill the table with your results**

1. Dense neural network

| H0 | H1 | batch size | RMSE |
|--  |--  |--          | --   |
| 16 | 8  | 32        | ... m/s |
| 128 | 64 | 32       | ... m/s |


2. Convolutional neural network

| Filters | H0 | H1 | batch size | RMSE |
|--       |--  |--  |--          |--    |
| 16      | 64 | 32 | 32         | ... m/s |
| 32      | 128 | 64 | 32        | ... m/s |

## <font color="#990099">Evaluation and roll-out</font>

Finally, we use our model to predict wind speeds for the test set. So far, we did not touch this dataset, so it should give an idea how well our machine learning algorithm generalizes to unseen data. We repeat the dataset preparation for the test set

In [ ]:
X_test, y_test = create_dataset('test', network='dense')
X_test_cnn, _ = create_dataset('test',  network='cnn')

Choose one of the model architectures that you think perform well on the given dataset. If necessary, train this model again. 

In [ ]:
# Here: Train again if necessary
best_model = model2

Use the trained model to make predictions on the test set. Un-comment the line corresponding to your best architecture.

In [ ]:
#y_pred = best_model.predict(X_test_cnn) # if CNN was best model
y_pred = best_model.predict(X_test) # if ANN was best model

## Data visualization

Calculate metrics to report on the performance of your machine learning algorithm. Compare the test set RMSE with the validation RMSE. 

**Discuss: What do you observe? Is your RMSE on the test larger or smaller than the RMSE obtained on the validation set?**

In [ ]:
rmse = mean_squared_error(y_test, y_pred, squared=False)

print(f'Root mean squared error (RMSE) for the test set: {rmse:.4} m/s')

### Histogram plot

In a regression problem, it is interesting to see the performance of the machine learning algorithm beyond the aggregated RMSE metric. We plot the histogram of true windspeed and predicted windspeed. What do you observe? Can you identify a windspeed range where our machine learning algorithm performs poorly? What are possible explanations?

In [ ]:
fig, ax = plt.subplots(1, 1)

sns.distplot(y_test.squeeze(), color='gray', label='True wind speed', ax=ax)
sns.distplot(y_pred.squeeze(), color='C2', label='Predicted wind speed', ax=ax)

ax.legend()
ax.set_xlabel('Wind speed (m/s)')

plt.show()

In [ ]:
# 2D scatter plot

fig, ax = plt.subplots(1, 1)

ax.set_aspect('equal')

img = ax.hexbin(y_test.squeeze(), y_pred.squeeze(), mincnt=1, cmap='viridis')
plt.colorbar(img, label='Sample density')

ax.set_xlabel('ERA5 wind speed (m/s)')
ax.set_ylabel('Predicted wind speed (m/s)')

xmin = 0
xmax = 20

ax.plot(np.linspace(xmin, xmax), np.linspace(xmin, xmax), 'r--')

ax.set_ylim(xmin, xmax)
ax.set_xlim(xmin, xmax)


ax.set_xticks(np.arange(xmin, xmax + 2.5, 2.5))
ax.set_yticks(np.arange(xmin, xmax + 2.5, 2.5))

plt.show()

**Discuss: What do you think of the network's performance? What could be done to increase the performance? Can you identify samples that are harder to predict correctly than others?** 

*Related publication:*

Asgarimehr, M., Arnold, C., Weigel, T., Ruf, C. & Wickert, J. GNSS reflectometry global ocean wind speed using deep learning: Development and assessment of CyGNSSnet. Remote Sensing of Environment 269, 112801 (2022).

    
    